### **0. Set-up**

In [1]:
# Import libraries and utils
%run '../../utils.ipynb'

In [2]:
# Get api key
load_dotenv()
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')

# Set client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

In [3]:
# Load dataframe
en_simlex = pd.read_csv("../../../data/dataset/cleaned-nl-simlex-999.csv")

# Convert to tuple
tuples_list = list(zip(en_simlex['word1'], en_simlex['word2']))

In [4]:
# Show results
tuples_list

[('oud', 'nieuw'),
 ('slim', 'intelligent'),
 ('hard', 'moeilijk'),
 ('gelukkig', 'vrolijk'),
 ('hard', 'stoer'),
 ('snel', 'razendsnel'),
 ('gelukkig', 'blij'),
 ('kort', 'lang'),
 ('dom', 'stom'),
 ('vreemd', 'eigenaardig'),
 ('breed', 'smal'),
 ('makkelijk', 'moeilijk'),
 ('moeilijk', 'gemakkelijk'),
 ('slim', 'dom'),
 ('krankzinnig', 'gek'),
 ('gelukkig', 'kwaad'),
 ('uitgebreid', 'groot'),
 ('moeilijk', 'simpel'),
 ('nieuw', 'vers'),
 ('scherp', 'saai'),
 ('vlug', 'snel'),
 ('dom', 'dwaas'),
 ('prachtig', 'fantastisch'),
 ('eigenaardig', 'vreemd'),
 ('gelukkig', 'boos'),
 ('smal', 'breed'),
 ('eenvoudig', 'gemakkelijk'),
 ('oud', 'vers'),
 ('kennelijk', 'duidelijk'),
 ('betaalbaar', 'goedkoop'),
 ('leuk', 'grootmoedig'),
 ('raar', 'vreemd'),
 ('vreemd', 'normaal'),
 ('slecht', 'immoreel'),
 ('verdrietig', 'grappig'),
 ('prachtig', 'geweldig'),
 ('schuldig', 'beschaamd'),
 ('mooi', 'prachtig'),
 ('zelfverzekerd', 'zeker'),
 ('dom', 'onderontwikkeld'),
 ('groot', 'flexibel'),
 ('aar

### **1. Define and Evaluate Parameters**

In [5]:
# Define prompt
prompt = ("Rate the semantic similarity of each word pair on a scale from 0 to 10, "
          "where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. "
          "Use two decimals. The response should strictly adhere to the structure: "
          "[('word1', 'word2', <score>), ('word3', 'word4', <score>), ...]. "
          "Do not provide additional explanations or context.")

In [6]:
# Define model
model = "meta-llama/llama-3.3-70b-instruct"

# Set sample size
sample_size = 15

# Delay between individual API calls
delay = 15.0

# Define number of sublists
n_sublists = 20

In [7]:
# Split list
chunks = split_into_n_lists(tuples_list, n_sublists)

# Count number of lists
print(len(chunks))

20


In [8]:
# Print prompts for each chunk
print_prompts(chunks, prompt)

Rate the semantic similarity of each word pair on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>), ('word3', 'word4', <score>), ...]. Do not provide additional explanations or context. --- ["('oud', 'nieuw'), ('slim', 'intelligent'), ('hard', 'moeilijk'), ('gelukkig', 'vrolijk'), ('hard', 'stoer'), ('snel', 'razendsnel'), ('gelukkig', 'blij'), ('kort', 'lang'), ('dom', 'stom'), ('vreemd', 'eigenaardig'), ('breed', 'smal'), ('makkelijk', 'moeilijk'), ('moeilijk', 'gemakkelijk'), ('slim', 'dom'), ('krankzinnig', 'gek'), ('gelukkig', 'kwaad'), ('uitgebreid', 'groot'), ('moeilijk', 'simpel'), ('nieuw', 'vers'), ('scherp', 'saai'), ('vlug', 'snel'), ('dom', 'dwaas'), ('prachtig', 'fantastisch'), ('eigenaardig', 'vreemd'), ('gelukkig', 'boos'), ('smal', 'breed'), ('eenvoudig', 'gemakkelijk'), ('oud', 'vers'), ('kennelijk', 'duide

In [9]:
# Load encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens per chunk
token_counts = count_tokens_with_tiktoken(chunks, prompt)

# Show results
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [548, 539, 515, 489, 498, 509, 503, 516, 507, 509, 533, 539, 490, 511, 543, 532, 552, 558, 565, 557]


### **2. Extract and Process Data**

In [10]:
# Get results from API
response = get_responses(chunks, prompt, model, sample_size, delay)

Processing: 100%|██████████| 300/300 [3:26:00<00:00, 41.20s/chunk]  

Total time taken: 12360.16 seconds


In [ ]:
# Define filepath
file_path = '../../../data/llama-3.3-70b/response/nl/f1.json'

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File saved successfully.


In [35]:
# Process data into dictionary
data_dict = process_responses(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{('oud', 'nieuw'): [0.0, 0.0, 0.0, 2.0, 2.5, 2.5, 0.0, 2.0, 0.0, 2.1, 2.1, 2.1, 0.0], ('slim', 'intelligent'): [9.5, 9.8, 9.8, 9.0, 9.8, 9.8, 9.5, 9.5, 9.8, 9.5, 9.2, 9.5, 9.5], ('hard', 'moeilijk'): [7.2, 7.4, 7.5, 7.5, 8.2, 7.2, 7.5, 7.2, 7.5, 7.2, 7.8, 7.8, 7.2], ('gelukkig', 'vrolijk'): [8.8, 9.2, 9.2, 8.5, 9.5, 8.5, 9.0, 8.8, 9.4, 9.8, 8.5, 8.9, 8.8], ('hard', 'stoer'): [6.5, 6.5, 8.5, 6.0, 7.8, 8.0, 8.0, 6.5, 8.2, 8.5, 6.4, 8.2, 6.5], ('snel', 'razendsnel'): [9.0, 8.9, 9.5, 9.0, 9.2, 9.5, 9.2, 9.0, 9.9, 9.9, 9.5, 9.4, 9.0], ('gelukkig', 'blij'): [9.2, 9.5, 9.4, 9.0, 9.5, 9.2, 9.5, 9.2, 9.6, 9.9, 9.0, 9.3, 9.2], ('kort', 'lang'): [0.0, 0.0, 0.0, 1.0, 1.5, 1.0, 0.0, 1.0, 0.0, 1.0, 1.2, 1.4, 0.0], ('dom', 'stom'): [9.8, 9.7, 9.6, 9.0, 9.8, 9.5, 9.8, 8.5, 9.9, 9.4, 8.8, 9.1, 9.8], ('vreemd', 'eigenaardig'): [9.4, 9.4, 9.4, 9.5, 9.5, 9.0, 9.4, 9.0, 9.7, 9.6, 9.8, 9.2, 9.4], ('breed', 'smal'): [0.0, 0.0, 0.0, 1.5, 2.2, 2.0, 0.0, 1.5, 0.0, 1.2, 2.5, 2.5, 0.0], ('makkelijk', 'moeilijk'):

In [36]:
# Print duplicate word pairs
print_duplicate_word_pairs(en_simlex, data_dict)

Empty DataFrame
Columns: [Combined_Columns]
Index: []
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [37]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,oud,nieuw,0.0,0.0,0.0,2.0,2.5,2.5,0.0,2.0,0.0,2.1,2.1,2.1,0.0,NaN,NaN
1,slim,intelligent,9.5,9.8,9.8,9.0,9.8,9.8,9.5,9.5,9.8,9.5,9.2,9.5,9.5,NaN,NaN
2,hard,moeilijk,7.2,7.4,7.5,7.5,8.2,7.2,7.5,7.2,7.5,7.2,7.8,7.8,7.2,NaN,NaN
3,gelukkig,vrolijk,8.8,9.2,9.2,8.5,9.5,8.5,9.0,8.8,9.4,9.8,8.5,8.9,8.8,NaN,NaN
4,hard,stoer,6.5,6.5,8.5,6.0,7.8,8.0,8.0,6.5,8.2,8.5,6.4,8.2,6.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,samenvoegen,verwerven,8.0,8.6,7.6,8.2,8.3,8.6,8.5,8.0,8.5,7.2,7.8,7.6,8.5,7.2,8.7
1018,sturen,bijwonen,6.5,6.8,6.1,5.2,6.6,5.9,6.2,5.5,6.8,5.5,5.0,5.5,7.7,5.8,6.4
1019,verzamelen,bijwonen,7.2,7.5,8.3,7.4,7.2,7.8,7.8,7.6,8.0,6.7,6.2,8.3,8.2,7.6,8.3
1020,opnemen,intrekken,5.3,5.1,5.2,6.6,5.2,6.7,6.9,6.0,7.5,7.8,8.4,7.9,8.9,6.8,7.5


In [38]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                    0
word2                    0
similarity_score_1       0
similarity_score_2      25
similarity_score_3      25
similarity_score_4      25
similarity_score_5      25
similarity_score_6      25
similarity_score_7      25
similarity_score_8      25
similarity_score_9      25
similarity_score_10     25
similarity_score_11     25
similarity_score_12     25
similarity_score_13     39
similarity_score_14     75
similarity_score_15    100
dtype: int64


In [39]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,oud,nieuw,0.0,0.0,0.0,2.0,2.5,2.5,0.0,2.0,0.0,2.1,2.1,2.1,0.0,NaN,NaN
1,slim,intelligent,9.5,9.8,9.8,9.0,9.8,9.8,9.5,9.5,9.8,9.5,9.2,9.5,9.5,NaN,NaN
2,hard,moeilijk,7.2,7.4,7.5,7.5,8.2,7.2,7.5,7.2,7.5,7.2,7.8,7.8,7.2,NaN,NaN
3,gelukkig,vrolijk,8.8,9.2,9.2,8.5,9.5,8.5,9.0,8.8,9.4,9.8,8.5,8.9,8.8,NaN,NaN
4,hard,stoer,6.5,6.5,8.5,6.0,7.8,8.0,8.0,6.5,8.2,8.5,6.4,8.2,6.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,maken,erdienen,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
871,aankomen,ertrekken,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
872,toevoegen,erdelen,5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873,deelnemen,eedoen,9.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# Extract missing word pairs
missing_word_pair_list = list(zip(rows_with_null['word1'], rows_with_null['word2']))

# Show results
missing_word_pair_list

[('oud', 'nieuw'),
 ('slim', 'intelligent'),
 ('hard', 'moeilijk'),
 ('gelukkig', 'vrolijk'),
 ('hard', 'stoer'),
 ('snel', 'razendsnel'),
 ('gelukkig', 'blij'),
 ('kort', 'lang'),
 ('dom', 'stom'),
 ('vreemd', 'eigenaardig'),
 ('breed', 'smal'),
 ('makkelijk', 'moeilijk'),
 ('moeilijk', 'gemakkelijk'),
 ('slim', 'dom'),
 ('krankzinnig', 'gek'),
 ('gelukkig', 'kwaad'),
 ('uitgebreid', 'groot'),
 ('moeilijk', 'simpel'),
 ('nieuw', 'vers'),
 ('scherp', 'saai'),
 ('vlug', 'snel'),
 ('dom', 'dwaas'),
 ('prachtig', 'fantastisch'),
 ('eigenaardig', 'vreemd'),
 ('gelukkig', 'boos'),
 ('smal', 'breed'),
 ('eenvoudig', 'gemakkelijk'),
 ('oud', 'vers'),
 ('kennelijk', 'duidelijk'),
 ('betaalbaar', 'goedkoop'),
 ('leuk', 'grootmoedig'),
 ('raar', 'vreemd'),
 ('vreemd', 'normaal'),
 ('slecht', 'immoreel'),
 ('verdrietig', 'grappig'),
 ('prachtig', 'geweldig'),
 ('schuldig', 'beschaamd'),
 ('mooi', 'prachtig'),
 ('zelfverzekerd', 'zeker'),
 ('dom', 'onderontwikkeld'),
 ('groot', 'flexibel'),
 ('aar

In [ ]:
# Define file_path
file_path = '../../../data/prompt/nl/llama-3.3-70b/f1.csv'

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")